# ADM Quantities in terms of BSSN Quantities

### Author: Zach Etienne

This tutorial module constructs all quantities in the [ADM formalism](https://en.wikipedia.org/wiki/ADM_formalism) (see also Chapter 2 in Baumgarte & Shapiro's book *Numerical Relativity*) in terms of quantities in our adopted (covariant, tensor-rescaled) BSSN formalism. That is to say, we will write the ADM quantities $\left\{\gamma_{ij},K_{ij},\alpha,\beta^i\right\}$ and their derivatives in terms of the BSSN quantities $\left\{\bar{\gamma}_{ij},\text{cf},\bar{A}_{ij},\text{tr}K,\alpha,\beta^i\right\}$ and their derivatives.

## A Note on Notation

As is standard in NRPy+, 

* Greek indices refer to four-dimensional quantities where the zeroth component indicates temporal (time) component.
* Latin indices refer to three-dimensional quantities. This is somewhat counterintuitive since Python always indexes its lists starting from 0. As a result, the zeroth component of three-dimensional quantities will necessarily indicate the first *spatial* direction.

As a corollary, any expressions in NRPy+ involving mixed Greek and Latin indices will need to offset one set of indices by one; a Latin index in a four-vector will be incremented and a Greek index in a three-vector will be decremented (however, the latter case does not occur in this tutorial module).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#initializenrpy): Initialize core NRPy+ modules
1. [Step 2](#threemetric): The ADM three-metric $\gamma_{ij}$ and its derivatives in terms of rescaled BSSN quantities.
1. [Step 3](#extrinsiccurvature): The ADM extrinsic curvature $K_{ij}$ and its derivatives in terms of rescaled BSSN quantities.
1. [Step 4](#code_validation): Code validation; comparing this tutorial module's output with the corresponding NRPy+ Python module [BSSN.ADM_in_terms_of_BSSN](../BSSN/ADM_in_terms_of_BSSN.py).
1. [Step 5](#latex_pdf_output) Output this module to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1.a: import all needed modules from NRPy+:
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm

# Step 1.b: Set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Spherical")

# Step 1.c: Given the chosen coordinate system, set up 
#           corresponding reference metric and needed
#           reference metric quantities
# The following function call sets up the reference metric
#    and related quantities, including rescaling matrices ReDD,
#    ReU, and hatted quantities.
rfm.reference_metric()

# Step 1.d: Set spatial dimension (must be 3 for BSSN, as BSSN is 
#           a 3+1-dimensional decomposition of the general 
#           relativistic field equations)
DIM = 3

# Step 1.e: Import all basic (unrescaled) BSSN scalars & tensors
import BSSN.BSSN_quantities as Bq
Bq.BSSN_basic_tensors()
gammabarDD = Bq.gammabarDD
cf         = Bq.cf
AbarDD     = Bq.AbarDD
trK        = Bq.trK

<a id='threemetric'></a>

# Step 1: The ADM three-metric $\gamma_{ij}$ and its derivatives in terms of rescaled BSSN quantities. \[Back to [top](#toc)\]
$$\label{threemetric}$$

The ADM three-metric is written in terms of the covariant BSSN three-metric tensor as (Eqs. 2 and 3 of [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):
$$
\gamma_{ij} = \left(\frac{\gamma}{\bar{\gamma}}\right)^{1/3} \bar{\gamma}_{i j},
$$
where $\gamma=\det{\gamma_{ij}}$ and $\bar{\gamma}=\det{\bar{\gamma}_{ij}}$. 

The "standard" BSSN conformal factor $\phi$ is given by (Eq. 3 of [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):

\begin{align}
\phi &= \frac{1}{12} \log\left(\frac{\gamma}{\bar{\gamma}}\right) \\
\implies e^{\phi} &= \left(\frac{\gamma}{\bar{\gamma}}\right)^{1/12} \\
\implies e^{4 \phi} &= \left(\frac{\gamma}{\bar{\gamma}}\right)^{1/3}
\end{align}

Thus the ADM three-metric may be written in terms of the BSSN three-metric and conformal factor $\phi$ as

$$
\gamma_{ij} = e^{4 \phi} \bar{\gamma}_{i j}.
$$

NRPy+'s implementation of BSSN allows for $\phi$ and two other alternative conformal factors to be defined:

\begin{align}
\chi &= e^{-4\phi} \\
W &= e^{-2\phi},
\end{align}

Thus if "BSSN_quantities::ConformalFactor" is set to "chi", then

\begin{align}
\gamma_{ij} &= \frac{1}{\chi} \bar{\gamma}_{i j} \\
&= \frac{1}{\text{cf}} \bar{\gamma}_{i j},
\end{align}

and if "BSSN_quantities::ConformalFactor" is set to "W", then
\begin{align}
\gamma_{ij} &= \frac{1}{W^2} \bar{\gamma}_{i j} \\
&= \frac{1}{\text{cf}^2} \bar{\gamma}_{i j}.
\end{align}

In [2]:
gammaDD = ixp.zerorank2()

exp4phi = sp.sympify(0)
if par.parval_from_str("ConformalFactor") == "phi":
    exp4phi = sp.exp(4*cf)
elif par.parval_from_str("ConformalFactor") == "chi":
    exp4phi = (1 / cf)
elif par.parval_from_str("ConformalFactor") == "W":
    exp4phi = (1 / cf**2)
else:
    print("Error ConformalFactor type = \""+par.parval_from_str("ConformalFactor")+"\" unknown.")
    exit(1)            

for i in range(DIM):
    for j in range(DIM):
        gammaDD[i][j] = exp4phi*gammabarDD[i][j]

<a id='extrinsiccurvature'></a>

# Step 2: The ADM extrinsic curvature $K_{ij}$ and its derivatives in terms of rescaled BSSN quantities. \[Back to [top](#toc)\]
$$\label{extrinsiccurvature}$$

The ADM extrinsic curvature may be written in terms of the BSSN trace-free extrinsic curvature tensor $\bar{A}_{ij}$ and the trace of the ADM extrinsic curvature $K$:

\begin{align}
K_{ij} &= \left(\frac{\gamma}{\bar{\gamma}}\right)^{1/3} \bar{A}_{ij} + \frac{1}{3} \gamma_{ij} K \\
&= e^{4\phi} \bar{A}_{ij} + \frac{1}{3} \gamma_{ij} K \\
\end{align}

In [3]:
KDD = ixp.zerorank2()

for i in range(DIM):
    for j in range(DIM):
        KDD[i][j] = exp4phi*AbarDD[i][j] + sp.Rational(1,3)*gammaDD[i][j]*trK

<a id='code_validation'></a>

# Step 3: Code validation; comparing this tutorial module's output with the corresponding NRPy+ Python module [BSSN.ADM_in_terms_of_BSSN](../BSSN/ADM_in_terms_of_BSSN.py)
$$\label{code_validation}$$



<a id='latex_pdf_output'></a>

# Step 4: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ADM_in_terms_of_BSSN.ipynb
!pdflatex -interaction=batchmode Tutorial-ADM_in_terms_of_BSSN.tex
!pdflatex -interaction=batchmode Tutorial-ADM_in_terms_of_BSSN.tex
!pdflatex -interaction=batchmode Tutorial-ADM_in_terms_of_BSSN.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ADM_in_terms_of_BSSN.ipynb to latex
